# lstm 做文本分类

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import pandas as pd

## 1数据处理     
bow模型并不适用于这类场景

In [2]:
bow_cut=10000
emb_size=64
lstm_size=64
batch_size=64
seq_cut=400

In [3]:
import nltk
from nltk.corpus import stopwords
import collections
import os
import codecs

In [4]:
pos_list=[]
with open('../data/Imdb/pos_all.txt','r',encoding='utf8')as f:
    line=f.readlines()
    pos_list.extend(line)
neg_list=[]
with open('../data/Imdb/neg_all.txt','r',encoding='utf8')as f:
    line=f.readlines()
    neg_list.extend(line)

pos_list = pos_list[:bow_cut]
neg_list = neg_list[:bow_cut]

#创建标签
label=[1 for i in range(bow_cut)]
label.extend([0 for i in range(bow_cut)])
#评论内容整合
pos_list.extend(neg_list)
content=pos_list
print(len(content),len(label))

20000 20000


In [5]:
label

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [6]:
seq=[]
seqtence=[]
stop_words={}
for con in content:
    words=nltk.word_tokenize(con)    
    line=[]
    for word in words:
        if word.isalpha() and word not in stop_words:
            line.append(word)   
    seq.append(line)
    seqtence.extend(line)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/qianlai/nltk_data'
    - '/opt/anaconda3/nltk_data'
    - '/opt/anaconda3/share/nltk_data'
    - '/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [90]:
print(len(seq),len(seqtence))

20000 2592518


In [91]:
vocab = set(seqtence)
print("词表大小",len(vocab))
vocab_to_int = {c: i+1 for i, c in enumerate(vocab)}
int_to_vocab = {i+1: c for i, c in enumerate(vocab)}


词表大小 80809


In [92]:
encoded = []
for a_seq in seq:
    a_encoded = [vocab_to_int[c] for c in a_seq]
    if len(a_encoded)>seq_cut:
        a_encoded=a_encoded[:seq_cut]
    else:
        a_encoded=np.pad(a_encoded, (0,seq_cut-len(a_encoded)), 'constant', constant_values=0)
    encoded.append(a_encoded)
encoded=np.array(encoded)
encoded.shape

(20000, 400)

In [93]:
label=np.array(label)
label.shape

(20000,)

In [95]:
encoded

array([[41139,  8319, 45647, ...,     0,     0,     0],
       [60922, 45901,  2091, ...,     0,     0,     0],
       [ 8968,  1336, 30216, ...,     0,     0,     0],
       ...,
       [46564,  3300, 52547, ...,     0,     0,     0],
       [46564,  3300, 78083, ...,     0,     0,     0],
       [46564,  3300, 62169, ...,     0,     0,     0]])

In [96]:
label

array([1, 1, 1, ..., 0, 0, 0])

## 2模型构建

In [97]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(vocab)+1, emb_size),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_size)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 64)          5171840   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 5,246,209
Trainable params: 5,246,209
Non-trainable params: 0
_________________________________________________________________


In [98]:
history = model.fit(encoded,label,batch_size=batch_size, epochs=2,validation_split=0.2)


Train on 16000 samples, validate on 4000 samples
Epoch 1/2
11776/16000 [=====================>........] - ETA: 16:18 - loss: 0.6956 - accuracy: 0.406 - ETA: 9:53 - loss: 0.6955 - accuracy: 0.382 - ETA: 7:40 - loss: 0.6954 - accuracy: 0.36 - ETA: 6:33 - loss: 0.6953 - accuracy: 0.35 - ETA: 5:53 - loss: 0.6949 - accuracy: 0.38 - ETA: 5:26 - loss: 0.6947 - accuracy: 0.39 - ETA: 5:07 - loss: 0.6945 - accuracy: 0.40 - ETA: 4:51 - loss: 0.6942 - accuracy: 0.42 - ETA: 4:39 - loss: 0.6940 - accuracy: 0.45 - ETA: 4:30 - loss: 0.6938 - accuracy: 0.47 - ETA: 4:23 - loss: 0.6936 - accuracy: 0.49 - ETA: 4:16 - loss: 0.6933 - accuracy: 0.50 - ETA: 4:11 - loss: 0.6931 - accuracy: 0.51 - ETA: 4:06 - loss: 0.6930 - accuracy: 0.52 - ETA: 4:00 - loss: 0.6928 - accuracy: 0.52 - ETA: 3:56 - loss: 0.6927 - accuracy: 0.53 - ETA: 3:53 - loss: 0.6924 - accuracy: 0.53 - ETA: 3:49 - loss: 0.6922 - accuracy: 0.54 - ETA: 3:46 - loss: 0.6921 - accuracy: 0.54 - ETA: 3:42 - loss: 0.6919 - accuracy: 0.54 - ETA: 3:40 -

11904/16000 [=====================>........] - ETA: 3:03 - loss: 0.5531 - accuracy: 0.62 - ETA: 3:08 - loss: 0.5546 - accuracy: 0.60 - ETA: 3:09 - loss: 0.5432 - accuracy: 0.63 - ETA: 3:10 - loss: 0.5437 - accuracy: 0.63 - ETA: 3:09 - loss: 0.5218 - accuracy: 0.65 - ETA: 3:09 - loss: 0.5456 - accuracy: 0.64 - ETA: 3:07 - loss: 0.5475 - accuracy: 0.64 - ETA: 3:07 - loss: 0.5517 - accuracy: 0.62 - ETA: 3:07 - loss: 0.5459 - accuracy: 0.63 - ETA: 3:06 - loss: 0.5483 - accuracy: 0.63 - ETA: 3:05 - loss: 0.5509 - accuracy: 0.62 - ETA: 3:05 - loss: 0.5510 - accuracy: 0.63 - ETA: 3:04 - loss: 0.5505 - accuracy: 0.63 - ETA: 3:03 - loss: 0.5467 - accuracy: 0.64 - ETA: 3:03 - loss: 0.5477 - accuracy: 0.65 - ETA: 3:02 - loss: 0.5481 - accuracy: 0.65 - ETA: 3:01 - loss: 0.5496 - accuracy: 0.65 - ETA: 3:01 - loss: 0.5473 - accuracy: 0.65 - ETA: 3:00 - loss: 0.5472 - accuracy: 0.66 - ETA: 2:59 - loss: 0.5497 - accuracy: 0.66 - ETA: 2:59 - loss: 0.5504 - accuracy: 0.66 - ETA: 2:58 - loss: 0.5495 - ac

16000/16000 [==============================] - ETA: 51s - loss: 0.4348 - accuracy: 0.814 - ETA: 50s - loss: 0.4337 - accuracy: 0.815 - ETA: 49s - loss: 0.4334 - accuracy: 0.815 - ETA: 48s - loss: 0.4330 - accuracy: 0.815 - ETA: 47s - loss: 0.4325 - accuracy: 0.815 - ETA: 47s - loss: 0.4320 - accuracy: 0.816 - ETA: 46s - loss: 0.4320 - accuracy: 0.815 - ETA: 45s - loss: 0.4317 - accuracy: 0.816 - ETA: 44s - loss: 0.4313 - accuracy: 0.816 - ETA: 43s - loss: 0.4305 - accuracy: 0.817 - ETA: 42s - loss: 0.4298 - accuracy: 0.817 - ETA: 42s - loss: 0.4304 - accuracy: 0.817 - ETA: 41s - loss: 0.4297 - accuracy: 0.817 - ETA: 40s - loss: 0.4288 - accuracy: 0.818 - ETA: 39s - loss: 0.4283 - accuracy: 0.818 - ETA: 38s - loss: 0.4277 - accuracy: 0.818 - ETA: 38s - loss: 0.4271 - accuracy: 0.819 - ETA: 37s - loss: 0.4266 - accuracy: 0.819 - ETA: 36s - loss: 0.4267 - accuracy: 0.819 - ETA: 35s - loss: 0.4259 - accuracy: 0.819 - ETA: 34s - loss: 0.4254 - accuracy: 0.820 - ETA: 34s - loss: 0.4250 - acc

In [99]:
model.save('./model/imdb_keras_lstm_classifier_epochs2.h5')